In [1]:
# Import libraries
import os
import json
import pandas as pd
import numpy as np
import nltk
import gensim
import re
import torch
import torchvision
from sklearn.model_selection import train_test_split
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from gensim.models import Word2Vec
from transformers import AutoTokenizer, AutoModelForSequenceClassification, AutoConfig
from datasets import load_dataset, load_from_disk
from collections import Counter

# # Download NLTK resources
# nltk.download('punkt_tab')
# nltk.download('stopwords')
# nltk.download('wordnet')

In [2]:
# Load the LIAR dataset
dataset = load_dataset("chengxuphd/liar2")

pretrained_model = "./without_icl/aug_roberta_1ep"

tokenizer = AutoTokenizer.from_pretrained(pretrained_model)

# Combine metadata with the preprocessed statement
def preprocess_function(examples):
    combined_input = [
        "Subject: " + (subject if subject is not None else "") + 
        "; Speaker: " + (speaker if speaker is not None else "") + 
        "; Speaker Description: " + (speaker_description if speaker_description is not None else "") + 
        "; State: " + (state_info if state_info is not None else "") + 
        "; Context: " + (context if context is not None else "") + 
        "; Statement: " + (statement if statement is not None else "")  # Don't apply preprocess_text here
        # "; Statement: " + preprocess_text(statement if statement is not None else "")  # Apply preprocess_text here
        for subject, speaker, speaker_description, state_info, context, statement in zip(
            examples["subject"],
            examples["speaker"],
            examples["speaker_description"],
            examples["state_info"],
            examples["context"],
            examples["statement"]
        )
    ]
    return tokenizer(combined_input, padding="max_length", truncation=True)

# Apply tokenization to the dataset
tokenized_datasets = dataset.map(preprocess_function, batched=True)

# Remove unnecessary columns and set format for PyTorch
tokenized_datasets = tokenized_datasets.remove_columns(["id", "subject", "speaker", "speaker_description", "state_info", "context", "true_counts", "mostly_true_counts", "half_true_counts", "mostly_false_counts", "false_counts", "pants_on_fire_counts", "justification"])
tokenized_datasets.set_format("torch")

# Preprocess labels to binary True / False
label_to_binary = {
    # True = FAKE; False = REAL
    0: True,
    1: True,
    2: True,
    3: False,
    4: False,
    5: False
}

# Apply binary label preprocessing
tokenized_datasets = tokenized_datasets.map(
    lambda examples: {"label": [label_to_binary[int(label)] for label in examples["label"]]},
    batched=True
)

# Split into train, validation, and test sets
train_dataset = tokenized_datasets["train"]
val_dataset = tokenized_datasets["validation"]
test_dataset = tokenized_datasets["test"]

In [3]:
# Verify that dataset is preprocessed correctly

# Check label distribution in the train dataset
label_list = train_dataset["label"].tolist()
print("Label distribution in train dataset:", Counter(label_list))

Label distribution in train dataset: Counter({1: 10591, 0: 7778})


In [4]:
from transformers import TrainingArguments, Trainer, EarlyStoppingCallback
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

# Define compute_metrics function
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average="binary")
    acc = accuracy_score(labels, preds)
    return {
        "accuracy": acc,
        "f1": f1,
        "precision": precision,
        "recall": recall,
    }

# Define training arguments (epoch)
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=5,
    weight_decay=0.01,
    learning_rate=2e-5,
    lr_scheduler_type="cosine",
    warmup_steps=500,
    logging_dir="./logs",
    logging_steps=10,
    report_to="none",
    load_best_model_at_end=True,  # Load the best model at the end of training
    metric_for_best_model="recall",  # Specify the metric to monitor
    greater_is_better=True       # Specify if higher values of the metric are better
)

c:\Users\limyi\AppData\Local\Programs\Python\Python311\Lib\site-packages\transformers\training_args.py:1611: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [5]:
config = AutoConfig.from_pretrained(pretrained_model, num_labels=2, hidden_dropout_prob=0.2, attention_probs_dropout_prob=0.2)
model = AutoModelForSequenceClassification.from_pretrained(pretrained_model, config=config)
tokenizer = AutoTokenizer.from_pretrained(pretrained_model)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)
model.config.id2label = {0: "REAL", 1: "FAKE"}
model.config.label2id = {"REAL": 0, "FAKE": 1}
training_args.num_train_epochs = 5
training_args.learning_rate = 1.5e-5
print("Model is on:", next(model.parameters()).device)
print("Learning rate:", training_args.learning_rate)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=2)]
)

trainer.train()

Model is on: cuda:0
Learning rate: 1.5e-05


C:\Users\limyi\AppData\Local\Temp\ipykernel_29032\2669159880.py:13: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,0.569400,0.544713,0.716152,0.728560,0.812442,0.660377
2,0.554400,0.558817,0.724859,0.746592,0.796407,0.702642
3,0.515400,0.558888,0.734436,0.761532,0.789943,0.735094
4,0.470700,0.570707,0.733130,0.747216,0.823636,0.683774
5,0.429200,0.596431,0.734436,0.751629,0.816092,0.696604


TrainOutput(global_step=5745, training_loss=0.47434837857570933, metrics={'train_runtime': 4733.9447, 'train_samples_per_second': 19.401, 'train_steps_per_second': 1.214, 'total_flos': 2.41654348795392e+16, 'train_loss': 0.47434837857570933, 'epoch': 5.0})

In [6]:
trainer.evaluate()

{'eval_loss': 0.5588876605033875,
 'eval_accuracy': 0.7344362211580322,
 'eval_f1': 0.7615324472243941,
 'eval_precision': 0.7899432278994323,
 'eval_recall': 0.7350943396226415,
 'eval_runtime': 33.3368,
 'eval_samples_per_second': 68.903,
 'eval_steps_per_second': 4.32,
 'epoch': 5.0}

In [15]:
model = AutoModelForSequenceClassification.from_pretrained("./results/checkpoint-2298")
trainer.model = model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
trainer.model.to(device)

RobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.2, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.2, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
         

In [7]:
# Conduct testing on the test dataset
test_results = trainer.predict(test_dataset)

# Extract predictions and metrics
predictions = test_results.predictions.argmax(-1)  # Convert logits to class predictions
metrics = test_results.metrics  # Contains accuracy, F1, precision, recall, etc.

# Print metrics
print("Test Metrics:")
for key, value in metrics.items():
    print(f"{key}: {value:.4f}")

Test Metrics:
test_loss: 0.5444
test_accuracy: 0.7326
test_f1: 0.7607
test_precision: 0.7852
test_recall: 0.7377
test_runtime: 33.6872
test_samples_per_second: 68.1560
test_steps_per_second: 4.2750


In [8]:
trainer.save_model("./without_icl/aug_norm_roberta")

### First trained on paraphrased statements, then original statements:

#### Before training on original statements:

| Metric                   | Value     |
|--------------------------|-----------|
| Test Loss                | 0.5404    |
| Test Accuracy            | 0.7086    |
| Test F1 Score            | 0.7396    |
| Test Precision           | 0.7624    |
| Test Recall              | 0.7181    |
| Test Runtime (seconds)   | 33.4549   |
| Test Samples/Second      | 68.6300   |
| Test Steps/Second        | 4.3040    |

#### After training on original statements:

| Metric                   | Value     |
|--------------------------|-----------|
| Test Loss                | 0.5444    |
| Test Accuracy            | 0.7326    |
| Test F1 Score            | 0.7607    |
| Test Precision           | 0.7852    |
| Test Recall              | 0.7377    |
| Test Runtime (seconds)   | 33.6872   |
| Test Samples/Second      | 68.1560   |
| Test Steps/Second        | 4.2750    |
